In [14]:
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
import polars as pl
from dotenv import load_dotenv
load_dotenv()
import utils
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
model = AzureChatOpenAI(
    deployment_name= "gpt-4o",
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    api_version="2023-09-15-preview",
    azure_endpoint="https://acsstscdamoai02.openai.azure.com/"
)

In [16]:
df_offers = pl.read_csv('sample_offers.csv')
df_offers.head()

offer,price_per_unit,quantity,bundling_unit,bundling_amount,bundling_discount,payment_term,payment_term_markup,delivery_timeline,contract_period,contract_inflation_by_year,rebates_threshold_unit,rebates_discount,warranty,incoterms
str,i64,i64,str,i64,i64,str,i64,i64,i64,str,i64,i64,i64,str
"""Price per unit: $95 Quantity: …",95,10000,null,20000,5,"""NET60""",3,10,2,"""[0,3]""",12000,2,1,"""DDP"""
"""Price per unit: $100 Quantity:…",100,10000,null,null,null,"""NET10""",0,7,1,"""[0]""",11000,1,1,"""FOB"""
"""I want to offer product for 13…",130,null,null,null,null,null,null,null,null,null,null,null,null,null


#### Logic Flows
* Extract levers from negotiation chat
* If there are any lever values present, negotiate and generate revised offer based on revised TCO
* If not then check if there are any lever mentioned in chat on which Supplier wants to negotiate
* If yes, then generate revised offer based on those levers.
* If no, then check if supplier is interested in negotiation.
* If yes then ask for levers which supplier want to negotiate
* If no then check if Supplier agreed with offer.
* If yes no then end conversation

In [19]:
utils.extract_levers_from_text(
    offer_text='quantity:300',
    model=model,
    example_data=df_offers
)

{'price_per_unit': None,
 'quantity': 300,
 'bundling_unit': None,
 'bundling_amount': None,
 'bundling_discount': None,
 'payment_term': None,
 'payment_term_markup': None,
 'delivery_timeline': None,
 'contract_period': None,
 'contract_inflation_by_year': None,
 'rebates_threshold_unit': None,
 'rebates_discount': None,
 'warranty': None,
 'incoterms': None}

#### Extract Lever(s) Supplier Wants to Negotiate

In [5]:
utils.understand_levers_to_negotiate(
    offer_text='I think price is too low',
    model = model
)

['price_per_unit']

#### If Supplier Wants to Negotiate But Did not Mention Any Lever: Generate Offer Revising Random Lever Variable

In [6]:
import random
from scipy.optimize import minimize_scalar

offer = utils.ContractOffer(
    price_per_unit = 100,
    quantity = 10000,
    bundling_unit = None,
    bundling_amount = None,
    bundling_discount = None,
    payment_term=utils.Payment(
        term=utils.PaymentTerm.NET10,
        markup=0
    ),
    delivery_timeline = 7,
    contract_period = 1,
    contract_inflation_by_year = [0],
    rebates_threshold_unit = 11000,
    rebates_discount = 1,
    warranty = 1,
    incoterms = utils.Incoterms.FOB
)
actual = utils.ContractActual(
    price_per_unit=90,
    quantity = 12000,
    payment_term=utils.PaymentTerm.NET30,
    incoterms=utils.Incoterms.FOB
)
print(f'Existing TCO: {actual.calculate_TCO_yearly(offer=offer)}')
utils.generate_offer_given_TCO_target(
    actual = actual,
    offer = offer,
    TCO_target = 1183000,
    lever_priority = {
        'quantity': 0.4,
        'price_per_unit': 0.4,
        'payment_term': 0.1,
        'incoterm':0.1
    },
    min_lever = {
        'quantity': 10000,
        'price_per_unit': 90
    },
    max_lever = {
        'quantity': 15000,
        'price_per_unit': 150
    }
)

Existing TCO: 1082442.410958904
Randonmly selected lever: price_per_unit


ContractActual(price_per_unit=98.47838088532492, quantity=12000, bundling_ind=None, payment_term=<PaymentTerm.NET30: 30>, delivery_timeline=None, contract_period=None, warranty=None, incoterms=<Incoterms.FOB: 15000>)

#### If Supplier Wants to Negotiate Lever(s): Generate Offer Accordingly

In [7]:
actual = utils.generate_offer_tuning_levers(
    offer = offer,
    actual=actual,
    tco_hike_pct=10,
    levers=['quantity','price_per_unit'],
    min_lever={
        'quantity': 10000,
        'price_per_unit': 90
    },
    max_lever={
        'quantity': 15000,
        'price_per_unit': 110
    }
)

Randonmly selected lever: quantity
Randonmly selected lever: price_per_unit


#### If Supplier Chat Was Neutral: Ask Which lever Supplier Wants to Negotiate

In [8]:
# negotiation_text = '''I don't think we will be able to make deal'''
negotiation_text = '''I think offer is too low'''
utils.is_interested_to_negotiate(
    offer_text = negotiation_text,
    model = model
)

True

#### If Supplier Does Not Want to Negotiate, End Negotiation Chat

In [11]:
from langchain_core.messages import SystemMessage, trim_messages
from langchain_core.messages import HumanMessage,AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

trimmer = trim_messages(
    max_tokens=10000,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

workflow = StateGraph(state_schema=State)

def call_model(state: State):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                '''You are an experienced Prefessional. Your job is to 
                negotiate with Suppliers. Always be polite and keep 
                professional tone throught the conversation''',
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    chain = prompt | model
    trimmed_messages = trimmer.invoke(state["messages"])
    response = chain.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [43]:
# Sample dictionary
data = {'a': None, 'b': None, 'c': None}

# Check if all values in the dictionary are None
all_null = all(value is None for value in data.values())

print("All values are None:", all_null)

All values are None: True


In [48]:
LEVER_PRIORITY = {
    'quantity': 0.4,
    'price_per_unit': 0.4,
    'payment_term': 0.1,
    'incoterm':0.1
}

In [58]:
# Extract levers from negotiation chat
# If there are any lever values present, negotiate and generate revised offer based on revised TCO
# If not then check if there are any lever mentioned in chat on which Supplier wants to negotiate
# If yes, then generate revised offer based on those levers.
# If no, then check if supplier is interested in negotiation.
# If yes then ask for levers which supplier want to negotiate
# If no then end the conversation
existing_offer = '''
Price per unit: $95
Quantity: 10,000 units
Bundling: If ordered with additional safety gear ($20,000 total), a 5% discount on the total contract.
Payment Terms: NET30 with a 3% markup if NET60 is requested.
Delivery Timelines: 10 days.
Contract Period Length: 2 years with a 3% price increase in the second year.
Rebates: 2% rebate if the purchase volume exceeds 12,000 units in the first year.
Warranties: 1-year standard warranty.
Incoterms: DDP (Delivered Duty Paid)
'''
supplier_msg = '''This deal is nearly impossible'''


existing_offer = utils.ContractOffer().load_from_data(
    utils.extract_levers_from_text(
        offer_text=existing_offer,
        model= model,
        example_data=df_offers
    )
)
print('Existing offer: ',existing_offer)
actual = utils.ContractActual(
    price_per_unit=existing_offer.price_per_unit,
    quantity=existing_offer.quantity,
    payment_term=existing_offer.payment_term.term,
    incoterms=existing_offer.incoterms
)
tco = actual.calculate_TCO_yearly(existing_offer)
print('Existing TCO: ',tco)

levers = utils.extract_levers_from_text(
    offer_text = supplier_msg,
    model = model,
    example_data = df_offers
)
if all(value is None for value in levers.values()) == False:
    supplier_offer = actual.update_from_data(levers)
    print('Supplier offer: ',supplier_offer)

    tco_supplier = supplier_offer.calculate_TCO_yearly(existing_offer)
    print('TCO based on supplier offer: ',tco_supplier)

    tco_new = tco*1.1
    levers_to_negotiate = [lever for lever in levers if levers[lever] != None]
    print('Negotiation lever: ',levers_to_negotiate)

    revised_offer = utils.generate_offer_tuning_levers(
        offer=existing_offer,
        actual=actual,
        tco_hike_pct=10,
        levers=levers_to_negotiate,
        min_lever={'price_per_unit':actual.price_per_unit,'quantity':actual.quantity},
        max_lever={'price_per_unit':actual.price_per_unit*1.5,'quantity':actual.quantity*1.5}
    )
    print('Revised offer: ',revised_offer)

    if tco_new < tco_supplier:
        print('Share revised offer to supplier')
    else:
        print('Accept supplier offer')
else:
    levers = utils.understand_levers_to_negotiate(
        offer_text=supplier_msg,
        model = model
    )
    if len(levers) != 0:
        print('Levers to negotiate: ',levers)
        revised_offer = actual = utils.generate_offer_tuning_levers(
            offer=existing_offer,
            actual=actual,
            tco_hike_pct=10,
            levers=levers,
            min_lever={'price_per_unit':actual.price_per_unit,'quantity':actual.quantity},
            max_lever={'price_per_unit':actual.price_per_unit*1.5,'quantity':actual.quantity*1.5}
        )
        print('Revised offer: ',revised_offer)
    else:
        is_interested = utils.is_interested_to_negotiate(
            offer_text=supplier_msg,
            model = model
        )
        if is_interested:
            print('Supplier is interested to negotiate')
            revised_offer = utils.generate_offer_given_TCO_target(
                offer=existing_offer,
                actual=actual,
                TCO_target=tco*1.1,
                lever_priority=LEVER_PRIORITY,
                min_lever={'price_per_unit':actual.price_per_unit,'quantity':actual.quantity},
                max_lever={'price_per_unit':actual.price_per_unit*1.5,'quantity':actual.quantity*1.5}
            )
            print('Revised Offer: ',revised_offer)
        else:
            is_accepted = utils.is_offer_accepted(
                offer_text=supplier_msg,
                model = model
            )
            if is_accepted:
                print('Offer Accepted')
            else:
                print('Supplier does not want to negotiate')

Existing offer:  ContractOffer(price_per_unit=95, quantity=10000, bundling_unit=None, bundling_amount=20000, bundling_discount=5, payment_term=Payment(term=<PaymentTerm.NET60: 60>, markup=0), delivery_timeline=10, contract_period=2, contract_inflation_by_year=[0, 3], rebates_threshold_unit=12000, rebates_discount=2, warranty=1, incoterms=<Incoterms.DDP: 0>)
Existing TCO:  926097.401369863
Supplier does not want to negotiate


ContractOffer(price_per_unit=100, quantity=10000, bundling_unit=None, bundling_amount=None, bundling_discount=None, payment_term=Payment(term=<PaymentTerm.NET10: 10>, markup=0), delivery_timeline=7, contract_period=1, contract_inflation_by_year=[0], rebates_threshold_unit=11000, rebates_discount=1, warranty=1, incoterms=<Incoterms.FOB: 15000>)

In [31]:
revised_offer.calculate_TCO_yearly(existing_offer)

1379598.7061643838